# Decode Linear Page Address to Page, Block, Plane Index

In [27]:
def decode_page_address(page_address: int, pages_per_block: int, planes: int) -> tuple[int, int, int]:
    """
    Decode a linear page address into (page, block, plane_index)
    
    :param page_address:     The straight page address.
    :param pages_per_block:  Number of pages in each block.
    :param planes:           Number of planes (1, 2, 4, ...).
    
    :return: (page, block, plane_index)
    """
    # 1) Extract 'page' within a block:
    page = page_address % pages_per_block
    
    # 2) Remove 'page' bits to get the combined block/plane field:
    block_plane_field = page_address // pages_per_block
    
    # 3) Extract 'plane_index' from the lower bits:
    #    plane_index = (block_plane_field mod planes) 
    #    because if planes=2 => 1 bit; if planes=4 => 2 bits, etc.
    plane_index = block_plane_field % planes
    
    # 4) "pure block" index :
    block = block_plane_field
    
    #    If you prefer to keep plane bits inside 'block', then do:
    # block = block_plane_field
    
    return (page, block, plane_index)

In [28]:
linearPageAddress = 100
pagesPerBlock = 1024
NumberOfPlane = 4

pageX, blockX, planeX = decode_page_address(linearPageAddress, pagesPerBlock, NumberOfPlane)

print(f"Address {linearPageAddress} => Page {pageX}, Block {blockX}, Plane {planeX}")

Address 100 => Page 100, Block 0, Plane 0


# BUILD ADDRESS (5 Cycles)

In [29]:
def build_5_cycle_address(column: int, page: int, block: int, plane: int, lun: int) -> list[int]:
    """
    Build 5 address cycles (bytes) according to the layout:
    1) Column address (16 bits):
       - Cycle 1 => column[7..0]
       - Cycle 2 => column[15..8]
    2) Page address (10 bits total):
       - Cycle 3 => page[7..0]        (lowest 8 bits)
       - Cycle 4 => page[9..8]       (bits [1..0])
    3) Plane (2 bits), embedded in BA[11..10]:
       - Cycle 4 bits [3..2] => plane[1..0]
    4) Block address (10 bits), combined with plane bits => total 12 bits:
       - Cycle 4 bits [7..4] => block[3..0]   (changed as requested)
       - Cycle 5 bits [5..0] => block[9..4]   (changed as requested)
    5) LUN (1 bit):
       - Cycle 5 bit [6] => lun
    :param column: up to 16 bits (0..65535)
    :param page:   up to 10 bits (0..1023)
    :param block:  up to 10 bits (0..1023) – plane bits separate
    :param plane:  0..3 (2 bits)
    :param lun:    0..1 (1 bit)
    :return: List of 5 bytes [cycle1..cycle5].
    """

    # ---------------------
    # Cycle 1 => lower 8 bits of column
    # Cycle 2 => upper 8 bits of column
    # ---------------------
    c1 = column & 0xFF
    c2 = (column >> 8) & 0xFF
    
    # ---------------------
    # Cycle 3 => lower 8 bits of page: page[7..0]
    # ---------------------
    c3 = page & 0xFF
    
    # ---------------------
    # Cycle 4 => c4[7..0]:
    #   bits[7..4] -> block[3..0] 
    #   bits[3..2] -> plane[1..0]
    #   bits[1..0] -> page[9..8]
    # ---------------------
    c4 = 0
    # block[3..0] => lower 4 bits of block
    c4 |= (block & 0xF) << 4
    # plane => 2 bits in [3..2]
    c4 |= (plane & 0x3) << 2
    # top 2 bits of page => page[9..8]
    c4 |= (page >> 8) & 0x3
    
    
    # ---------------------
    # Cycle 5 => c5[7..0]:
    #   bits[5..0] -> block[9..4]
    #   bit [6]    -> lun
    #   bit [7]    -> 0
    # ---------------------
    c5 = 0
    # block[9..4] => 6 bits
    c5 |= ((block >> 4) & 0x3F)
    # lun => bit [6]
    c5 |= ((lun & 0x1) << 6)
    
    return [c1, c2, c3, c4, c5]

In [30]:
column = 0
page   = pageX
block  = blockX
plane  = planeX
lun    = 0

print("Input Address Parameters:")
print(f" Column    : 0x{column:02X}  (bin: {column:08b})")
print(f" Page      : 0x{page:02X}    (bin: {page:08b})")
print(f" Block     : 0x{block:02X}   (bin: {block:08b})")
print(f" Plane     : 0x{plane:02X}   (bin: {plane:08b})")


cycles = build_5_cycle_address(column, page, block, plane, lun)
print("5 Address Cycles:")
for i, val in enumerate(cycles, start=1):
    print(f" Cycle {i}: 0x{val:02X}  (bin: {val:08b})")

print(f"Address Cycles: {cycles}")

Input Address Parameters:
 Column    : 0x00  (bin: 00000000)
 Page      : 0x64    (bin: 01100100)
 Block     : 0x00   (bin: 00000000)
 Plane     : 0x00   (bin: 00000000)
5 Address Cycles:
 Cycle 1: 0x00  (bin: 00000000)
 Cycle 2: 0x00  (bin: 00000000)
 Cycle 3: 0x64  (bin: 01100100)
 Cycle 4: 0x00  (bin: 00000000)
 Cycle 5: 0x00  (bin: 00000000)
Address Cycles: [0, 0, 100, 0, 0]


In [31]:
#Conversion to Charater Array:
dataCycle = ""
noAddressCycle = 5

for i in range(0, noAddressCycle, 1):
    dataCycle += chr(cycles[i])
    
print(f" Character Array of Address Cycle: {dataCycle}")

 Character Array of Address Cycle:   d  


# Commands and Read Write Functions

In [32]:
#essential functions
import usb.core
import usb.util
import time
from array import array as Array
from pyftdi.ftdi import *

In [33]:
NAND_CMD_READ_CYCLE1 = 0x00
NAND_CMD_READ_CYCLE2 = 0x30
NAND_CMD_RESET = 0xFF

ACTIVATE_COMMNAD_LATCH = 0x40
ACTIVATE_ADDRESS_LATCH = 0x80
DEACTIVATE_CA_LATCH = 0x00

In [34]:
def read_nand_command (cycle):
    #Command Cycle 1
    cmds1 = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0] 
    cmds1 += [NAND_CMD_READ_CYCLE1]
    ftdi.write_data(Array('B', cmds1))
    
    #Address Cycle 5 Consequtive
    cmds2 = [Ftdi.WRITE_EXTENDED, ACTIVATE_ADDRESS_LATCH, 0 , cycle[0]]
    for i in range(1,5,1):
        cmds2 += [Ftdi.WRITE_SHORT, 0, cycle[i]]
    ftdi.write_data(Array('B', cmds2))
    
    #Command Cycle 2
    cmds3 = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0]
    cmds3 += [NAND_CMD_READ_CYCLE2]
    ftdi.write_data(Array('B', cmds3))
    
    print(f"Command 1:  {cmds1}")
    print(f"Command 2:  {cmds2}")
    print(f"Command 3:  {cmds3}")

In [35]:
def read_nand_page_bytes(read_len):
    cmds = []
    
    #Data Read Cycle
    cmds += [ftdi.READ_EXTENDED, DEACTIVATE_CA_LATCH , 0]
    
    for i in range(1, read_len, 1):
        cmds += [Ftdi.READ_SHORT, 0]
        
    cmds.append(Ftdi.SEND_IMMEDIATE)
    ftdi.write_data(Array('B', cmds))
    data = ftdi.read_data_bytes(read_len)  
    print(f"Type of the page data {type(data[1])}")
    print(f"Type of the page data array {type(data)}")
    
    return bytes(data)

# FTDI Initialization

In [36]:
VENDOR_ID = 0x0403
PRODUCT_ID = 0x6010

# direction_mask: 1 means output, 0 means input
# value_mask:     1 sets the pin high, 0 sets it low
DIRECTION_MASK = 0xFF  # all upper 8 bits as output
VALUE_MASK = 0x00     # pattern to set the pins to

# Initialize FTDI
#Creates an instance or object of the Ftdi class and stored in the ftdi variable
ftdi = Ftdi()

#Open a new interface to the specified FTDI device.
#open() method establishes a connection to a specific FTDI device.
ftdi.open(VENDOR_ID, PRODUCT_ID, interface=1)

#Configures the FTDI device into MCU Host Bus Emulation mode
ftdi.set_bitmode(0x00, Ftdi.BitMode.MCU)  # MCU = 0x08 (MCU Host Bus Emulation mode)

#Disable the “divide-by-5” clock prescaler allows higher frequency operation
ftdi.write_data(Array('B', [Ftdi.DISABLE_CLK_DIV5]))

#the FTDI device waits (in milliseconds) 
#before sending any partially filled USB packet up to the host if the buffer is not yet full.
ftdi.set_latency_timer(16)


# This tells the FTDI chip to set upper 8 bits [ACBUS] with these direction and value settings
ftdi.write_data(bytes([Ftdi.SET_BITS_HIGH, VALUE_MASK, DIRECTION_MASK]))

#Any pending outbound data in the transmit buffer is cleared
ftdi.purge_buffers()

#Reset the the nand flash chip
cmdsRST = [Ftdi.WRITE_EXTENDED, ACTIVATE_COMMNAD_LATCH, 0, NAND_CMD_RESET]
print (cmdsRST)
ftdi.write_data(Array('B', cmdsRST))

[147, 64, 0, 255]


4

In [37]:
print(cycles)
read_nand_command(cycles)
#read 5000 bytes from a page
read_nand_page_bytes(5000)

[0, 0, 100, 0, 0]
Command 1:  [147, 64, 0, 0]
Command 2:  [147, 128, 0, 0, 146, 0, 0, 146, 0, 100, 146, 0, 0, 146, 0, 0]
Command 3:  [147, 64, 0, 48]
Type of the page data <class 'int'>
Type of the page data array <class 'bytearray'>


b'\xc1AFFAAGFAAAAFFAANFC\xc1fFAAF^AAFFAAFFAAFFAAFFAAFFAAFFa\xc1VFAAFNAAFFAAFFAAFFAAFFAAGF\xc1AFFAANFAAFFAIFFAaFFAAFFAAFFAEFFAA\xc6FA\xc1F\xc6EAFFAAFNAIFfAAFFEAFFAaF\xc6A\xc1FFAEFFAEGFAAFFAAFFACFFAAFFAAFFAAG\xc7ACFVAAFF\xc1\xc1FFAAFFaAGFAIFFAAFFA\xc1VFAAFFAAFFAAF\xc6AAFfAAFfEAFFQAFFA\xc1FFAAFFAAFFAAfFAAGFAAFFAaFFAAFFAAFFQAFFAA\xc6FAAFFAAFFAAG\xe6AAFFAAFGA\xc1FfAAFFAAFFAAFFAAFFA\xc3GFAAFFACVFAAFNAAF\xc7AA^FAAF\xceAAFFAAFFAAFFAAFFAAFGAaFFAAFVaAFFAAFFIAFFAAGFAAFF~FFAAFF\xc1AFF\xc1A\xc6FAAFFAAFF\xc1AFGCAFNACFFAAFFAAAAFFAAFGQAFFaAFFAA\xc6FAANfAAFNAAN\xc6A\xc1FFAAFFAAFF\xc1AFFAAFFAANFAAFFAAF\xc6A\xc1FFAAF\xc6AAFFAAfFAC\xc6FAA\xc6FAAFF\xc1AFFaA\xc6FAAFFAAFFAAFVAAFFAAGGAMFFAc\xc6F\xd1AffAAFFAAVFAAFFAAF\xc6AIFFAAFFACFFAAFFaAFgA\xc1F\xc6A\xc1FFAAFFAAFFQAGFAAF\xc6AQ\xe6F\xc1AFFAAFFAAVFAAFFAANFAAF\xc6EAgOA\xc1FvAAFFAAFFaA\xc6FAAFFAAFFAAFFEAF^IAFFCE\xc6fAAFFA\xc5FFAAFF\xc1AFVAAFFAAFFAAFGaAFFAAFFAAFGAEfFAAFFAcFNEAF\xc7AC\xceFAAfFAAFNAA\xc6FAAFFAAFGAAVFCAF\xc6CAVF\xc1Af\xc6\xc1aFFAAF\xc6AAFFAA\xc6FCAF

In [38]:
# Close the connection
ftdi.close()